# Vertex Weighting

Can vertex weight be used find signal vertices?

In [1]:
import numpy as np
import pandas as pd
from graspologic.datasets import load_mice
from hyppo.ksample import KSample
from statsmodels.stats.multitest import multipletests

In [2]:
# Load the full mouse dataset
mice = load_mice()

# Stack all adjacency matrices in a 3D numpy array
graphs = np.array(mice.graphs)

# Sort the connectomes and genotype labels so BTBR is first
label_indices = np.argsort(mice.labels).reshape(4, 8)
label_indices = label_indices[[1, 0, 2, 3]].reshape(-1)
labels = mice.labels[label_indices]
graphs = graphs[label_indices]

# Get sample parameters
n_subjects = mice.meta["n_subjects"]
n_vertices = mice.meta["n_vertices"]

In [3]:
# Split the set of graphs by genotype
btbr = graphs[labels == "BTBR"]
b6 = graphs[labels == "B6"]
cast = graphs[labels == "CAST"]
dba2 = graphs[labels == "DBA2"]

connectomes = [btbr, b6, cast, dba2]

In [4]:
vertex_pvals = []

for roi in range(332):

    # Get the (i,j)-th edge for each connectome
    samples = [genotype[:, roi, :] for genotype in connectomes]

    # Calculate the p-value for the (i,j)-th edge
    try:
        statistic, pvalue = KSample("Dcorr").test(*samples, reps=10000000)
    except ValueError:
        # A ValueError is thrown when any of the samples have equal edge
        # weights (i.e. one of the inputs has 0 variance)
        statistic = np.nan
        pvalue = 1

    vertex_pvals.append([roi, statistic, pvalue])

In [5]:
# Convert the nested list to a dataframe
signal_vertices = pd.DataFrame(vertex_pvals, columns=["ROI", "stat", "pvalue"])
signal_vertices.sort_values(by="pvalue", inplace=True, ignore_index=True)

# Correct p-values
reject, holm_pvalue, _, _ = multipletests(
    signal_vertices["pvalue"], alpha=0.05, method="holm"
)
signal_vertices["holm_pvalue"] = holm_pvalue
signal_vertices["significant"] = reject
signal_vertices.sort_values(by="holm_pvalue", inplace=True, ignore_index=True)
signal_vertices.head()

,ROI,stat,pvalue,holm_pvalue,significant
0,82,0.855475,9.993691e-08,0.000033,True
1,309,0.851650,1.064597e-07,0.000035,True
2,126,0.850944,1.077108e-07,0.000036,True
3,248,0.847117,1.147466e-07,0.000038,True
4,269,0.840477,1.280620e-07,0.000042,True


In [6]:
def lookup_roi_name(roi):
    roi -= 1
    hemisphere = "R" if roi // 166 else "L"
    roi = roi % 166
    structure = mice.atlas["Structure"].values[roi]
    structure = " ".join(structure.split("_"))
    return f"{structure} ({hemisphere})"

In [7]:
# Get the top 20 strongest signal edges
strong_signal_vertices = signal_vertices.head(20)
strong_signal_vertices["ROI"] = strong_signal_vertices["ROI"].apply(lookup_roi_name)
strong_signal_vertices

,ROI,stat,pvalue,holm_pvalue,significant
0,Inferior Colliculus (L),0.855475,9.993691e-08,0.000033,True
1,Ventral Spinocerebellar Tract (R),0.851650,1.064597e-07,0.000035,True
2,Ventral Hippocampal Commissure (L),0.850944,1.077108e-07,0.000036,True
3,Inferior Colliculus (R),0.847117,1.147466e-07,0.000038,True
4,Trigeminal Motor Nucleus (R),0.840477,1.280620e-07,0.000042,True
5,Anterior Pretectal Nucleus (R),0.838293,1.327723e-07,0.000043,True
6,Trigeminal Motor Nucleus (L),0.838242,1.328856e-07,0.000043,True
7,Ectorhinal Cortex (L),0.838199,1.329790e-07,0.000043,True
8,Pararubral Nucleus (L),0.829333,1.539851e-07,0.000050,True
9,Fastigial Medial Dorsolateral Nucleus of Cereb...,0.824864,1.658030e-07,0.000054,True
